# 🖼️ AI Photo Analyzer - Google Colab版

完全オンラインで写真をAI解析し、Mylio Photos用のXMPファイルを生成します。

## 必要なもの
- Google Cloud Vision API サービスアカウントキー（JSON）
- DeepL API キー
- 解析したい写真

## Step 1: 依存パッケージのインストール

In [ ]:
!pip install -q google-cloud-vision deepl pillow

## Step 2: Google Cloud認証設定

サービスアカウントのJSONファイルをアップロードしてください。

In [ ]:
import os
from google.colab import files

print("📁 Google Cloud サービスアカウントのJSONファイルをアップロードしてください:")
uploaded = files.upload()

json_file = list(uploaded.keys())[0]
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_file
print(f"✅ 認証設定完了: {json_file}")

## Step 3: DeepL API キー設定

In [ ]:
import getpass

deepl_key = getpass.getpass("🔑 DeepL APIキーを入力: ")
os.environ['DEEPL_API_KEY'] = deepl_key
print("✅ DeepL APIキー設定完了")

## Step 4: 写真のアップロード

解析したい写真（JPG, PNG, RAW等）をアップロードしてください。

In [ ]:
import os
from pathlib import Path

# ディレクトリ作成
!mkdir -p /content/photos /content/output /content/temp

print("📷 写真をアップロードしてください（複数可）:")
uploaded_photos = files.upload()

for name, data in uploaded_photos.items():
    path = f'/content/photos/{name}'
    with open(path, 'wb') as f:
        f.write(data)
    print(f"  ✅ {name}")

print(f"\n📊 アップロード完了: {len(uploaded_photos)} ファイル")

## Step 5: AI解析実行

In [ ]:
from google.cloud import vision
import deepl
from PIL import Image
from datetime import datetime
from pathlib import Path
import logging

logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger()

# 設定
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.heic', '.heif', '.webp'}
RAW_EXTENSIONS = {'.cr2', '.cr3', '.nef', '.arw', '.raf', '.orf', '.rw2', '.dng'}
RAW_LABELS_JA = ['RAW', 'RAW画像', '生データ']
RAW_LABELS_EN = ['RAW', 'RAW Image', 'Unprocessed']

# クライアント初期化
vision_client = vision.ImageAnnotatorClient()
translator = deepl.Translator(os.environ['DEEPL_API_KEY'])

def create_shrink(image_path, shrink_size=640):
    """縮小画像を作成"""
    try:
        shrink_path = Path('/content/temp') / f"{image_path.stem}_shrink.jpg"
        with Image.open(image_path) as img:
            if img.mode in ('RGBA', 'P'):
                img = img.convert('RGB')
            w, h = img.size
            if w <= h:
                new_size = (shrink_size, int(h * shrink_size / w)) if w > shrink_size else (w, h)
            else:
                new_size = (int(w * shrink_size / h), shrink_size) if h > shrink_size else (w, h)
            resized = img.resize(new_size, Image.Resampling.LANCZOS)
            resized.save(shrink_path, 'JPEG', quality=85)
        return shrink_path
    except Exception as e:
        logger.error(f"縮小失敗: {e}")
        return None

def analyze_image(image_path):
    """Vision APIで解析"""
    with open(image_path, 'rb') as f:
        content = f.read()
    image = vision.Image(content=content)
    response = vision_client.annotate_image({
        'image': image,
        'features': [
            {'type_': vision.Feature.Type.LABEL_DETECTION, 'max_results': 20},
            {'type_': vision.Feature.Type.LANDMARK_DETECTION, 'max_results': 5},
            {'type_': vision.Feature.Type.OBJECT_LOCALIZATION, 'max_results': 10},
        ],
    })
    labels = [l.description for l in response.label_annotations if l.score >= 0.7]
    labels += [l.description for l in response.landmark_annotations if l.score >= 0.7]
    labels += [o.name for o in response.localized_object_annotations if o.score >= 0.7]
    return list(dict.fromkeys(labels))  # 重複除去

def translate_labels(labels):
    """日本語に翻訳"""
    if not labels:
        return []
    try:
        results = translator.translate_text(labels, source_lang='EN', target_lang='JA')
        return [r.text for r in results]
    except:
        return labels

def generate_xmp(ja_labels, en_labels, is_raw=False, raw_ext=''):
    """XMPを生成"""
    date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
    keywords = []
    if is_raw:
        keywords.extend(RAW_LABELS_JA + RAW_LABELS_EN)
        if raw_ext:
            keywords.append(raw_ext.upper().lstrip('.'))
    keywords.extend(ja_labels + en_labels)
    keywords = list(dict.fromkeys(keywords))

    xmp = f'''<?xpacket begin="\ufeff" id="W5M0MpCehiHzreSzNTczkc9d"?>
<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="AI Photo Analyzer (Colab)">
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
    <rdf:Description rdf:about=""
      xmlns:dc="http://purl.org/dc/elements/1.1/"
      xmlns:xmp="http://ns.adobe.com/xap/1.0/"
      xmlns:lr="http://ns.adobe.com/lightroom/1.0/"
      xmp:ModifyDate="{date}"
      xmp:CreatorTool="AI Photo Analyzer (Colab)">
      <dc:subject>
        <rdf:Bag>
'''
    for kw in keywords:
        escaped = kw.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        xmp += f'          <rdf:li>{escaped}</rdf:li>\n'
    xmp += '''        </rdf:Bag>
      </dc:subject>
    </rdf:Description>
  </rdf:RDF>
</x:xmpmeta>
<?xpacket end="w"?>'''
    return xmp

# 処理実行
photos_dir = Path('/content/photos')
output_dir = Path('/content/output')
stats = {'processed': 0, 'raw': 0, 'failed': 0}

for photo in photos_dir.iterdir():
    suffix = photo.suffix.lower()
    if suffix not in IMAGE_EXTENSIONS and suffix not in RAW_EXTENSIONS:
        continue

    is_raw = suffix in RAW_EXTENSIONS
    logger.info(f"\n{'🔴 RAW' if is_raw else '📷'} 処理中: {photo.name}")

    # RAWの場合、JPEGを探す
    source = photo
    if is_raw:
        for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG']:
            candidate = photos_dir / f"{photo.stem}{ext}"
            if candidate.exists():
                source = candidate
                logger.info(f"  → {source.name} を解析に使用")
                break
        else:
            logger.warning(f"  ⚠️ 解析用JPEGが見つかりません")
            stats['failed'] += 1
            continue

    # 縮小
    shrink = create_shrink(source)
    if not shrink:
        stats['failed'] += 1
        continue

    try:
        # 解析
        en_labels = analyze_image(shrink)
        logger.info(f"  📊 検出: {len(en_labels)} ラベル")

        # 翻訳
        ja_labels = translate_labels(en_labels)

        # XMP生成
        xmp = generate_xmp(ja_labels, en_labels, is_raw, photo.suffix if is_raw else '')
        xmp_name = f"{photo.name}.xmp" if is_raw else f"{photo.stem}.xmp"
        xmp_path = output_dir / xmp_name
        xmp_path.write_text(xmp, encoding='utf-8')

        logger.info(f"  ✅ 生成: {xmp_name}")
        if ja_labels[:3]:
            logger.info(f"  🏷️ ラベル例: {', '.join(ja_labels[:3])}")

        stats['processed'] += 1
        if is_raw:
            stats['raw'] += 1

    except Exception as e:
        logger.error(f"  ❌ エラー: {e}")
        stats['failed'] += 1
    finally:
        if shrink and shrink.exists():
            shrink.unlink()

print(f"\n{'='*50}")
print(f"✅ 処理完了")
print(f"  - 成功: {stats['processed']} ({stats['raw']} RAW)")
print(f"  - 失敗: {stats['failed']}")

## Step 6: XMPファイルのダウンロード

In [ ]:
# ZIPにまとめてダウンロード
!cd /content && zip -r xmp_files.zip output/

print("\n📥 XMPファイルをダウンロード:")
files.download('/content/xmp_files.zip')

print("\n📋 使い方:")
print("1. ZIPを解凍")
print("2. XMPファイルを元の写真と同じフォルダに配置")
print("3. Mylio Photosで同期 → 日本語検索可能に！")

## 生成されたXMPの確認（オプション）

In [ ]:
# 生成されたXMPの一覧
!ls -la /content/output/

# 最初のXMPの内容を表示
import glob
xmps = glob.glob('/content/output/*.xmp')
if xmps:
    print(f"\n📄 {Path(xmps[0]).name} の内容:")
    print("="*50)
    print(open(xmps[0]).read())